# In this notebook will cover  data mining techiques unsing Theta Data and YahooFinance. 

In [4]:
!pip install thetadata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=ee7ed9cb3f9e0c75b8a086c74f8a78605da89e810b8752707a10f5f656cad398
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [5]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime
import time
import pandas as pd
from IPython.display import display
from datetime import date
from thetadata import ThetaClient, OptionReqType, OptionRight, DateRange

In [6]:
client = ThetaClient()  # We don't need to provide a username / password because this is a free request.

with client.connect():  # Make any requests for data inside this block. Requests made outside this block wont run.
    data = client.get_hist_option(
          req=OptionReqType.EOD,
          root="AAPL",
          exp=date(2022, 11, 25),
          strike=150,
          right=OptionRight.CALL,
          date_range=DateRange(date(2020, 10, 15), date(2022, 11, 27))
    )

# We are out of the client.connect() block, so we can no longer make requests.
display(data)  # displays the data. You can also do print(data).

If you require API support, feel free to join our discord server! http://discord.thetadata.us
------------------------------------------------------------------------------------------------
You are using the free version of Theta Data. You are currently limited to 20 requests / minute.
A data subscription can be purchased at https://thetadata.net. If you already have a ThetaData
subscription, specify the username and passwd parameters.
------------------------------------------------------------------------------------------------
Starting Theta Terminal [v0.9.7 REV: A]  --> Config: /root/ThetaData/ThetaTerminal/config_0.properties
Configuration file not found: created a default config file.
[MDDS] Attempting login as default
[MDDS] CONNECTED: [nj-a.thetadata.us:12001], Bundle: FREE, Permissions: End of Day(EOD)
[FPSS] Attempting login as default


,DataType.OPEN,DataType.HIGH,DataType.LOW,DataType.CLOSE,DataType.VOLUME,DataType.COUNT,DataType.DATE
0,6.22,6.25,6.15,6.25,5,5,2022-10-06
1,4.47,4.47,3.75,3.90,60,24,2022-10-07
2,3.93,4.44,3.33,4.22,168,50,2022-10-10
3,3.85,4.10,3.26,3.43,92,42,2022-10-11
4,3.67,3.78,3.25,3.30,87,35,2022-10-12
5,2.27,4.75,2.20,4.69,160,88,2022-10-13
6,4.75,4.75,3.05,3.14,109,36,2022-10-14
7,3.75,4.22,3.55,4.10,903,82,2022-10-17
8,4.95,5.47,3.40,4.40,208,73,2022-10-18
9,4.20,4.85,3.91,4.38,132,40,2022-10-19


In [7]:
def yf_get_chains(ticker):
    """
    Loads option chains data for a particular ticker from Yahoo Finance
    """

    underlying = yf.Ticker(ticker)  # addressing the underlying asset of an options
    expiration_dates = underlying.options  # and dates of expiration

    opt_data = pd.DataFrame()  # constructing the dataframe we will return

    for ed in expiration_dates:
        opt = underlying.option_chain(ed)  # getting data for a specific date

        calls, puts = opt.calls, opt.puts  # call and puts
        calls['optionType'], puts['optionType'] = 'call', 'put'  # adding types of options to the df

        ed_chain = pd.concat([calls, puts])
        ed_chain['expDate'] = pd.to_datetime(ed) + pd.DateOffset(hours=23, minutes=59, seconds=59)
        # adding date of expiration

        opt_data = pd.concat([opt_data, ed_chain])  # adding data to the main df

    opt_data['daysToExp'] = (opt_data.expDate - datetime.datetime.today()).dt.days + 1
    # additionally let's compute days until expiration date

    return opt_data

In [8]:
df = yf_get_chains("JPM")

In [9]:
df.head(20)

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,optionType,expDate,daysToExp
0,JPM230609C00075000,2023-05-25 17:33:34+00:00,75.0,60.35,64.70,66.35,0.000000,0.000000,NaN,1,1.828126,True,REGULAR,USD,call,2023-06-09 23:59:59,7
1,JPM230609C00100000,2023-06-02 18:50:39+00:00,100.0,41.37,40.10,41.05,-0.130001,-0.313256,1.0,3,1.140629,True,REGULAR,USD,call,2023-06-09 23:59:59,7
2,JPM230609C00110000,2023-05-24 17:45:13+00:00,110.0,25.90,29.90,31.25,0.000000,0.000000,NaN,1,0.855470,True,REGULAR,USD,call,2023-06-09 23:59:59,7
3,JPM230609C00115000,2023-06-01 19:03:45+00:00,115.0,23.22,24.70,26.45,0.000000,0.000000,5.0,10,0.718753,True,REGULAR,USD,call,2023-06-09 23:59:59,7
4,JPM230609C00120000,2023-06-02 19:45:19+00:00,120.0,20.95,20.30,21.00,3.850000,22.514622,7.0,2,0.644535,True,REGULAR,USD,call,2023-06-09 23:59:59,7
5,JPM230609C00123000,2023-04-28 19:48:59+00:00,123.0,16.26,14.10,14.45,0.000000,0.000000,1.0,1,0.000010,True,REGULAR,USD,call,2023-06-09 23:59:59,7
6,JPM230609C00124000,2023-05-23 17:07:12+00:00,124.0,14.41,16.20,16.90,0.000000,0.000000,2.0,2,0.642582,True,REGULAR,USD,call,2023-06-09 23:59:59,7
7,JPM230609C00125000,2023-06-01 13:47:39+00:00,125.0,11.86,15.35,16.05,0.000000,0.000000,5.0,12,0.529302,True,REGULAR,USD,call,2023-06-09 23:59:59,7
8,JPM230609C00126000,2023-05-25 15:39:52+00:00,126.0,9.66,14.35,15.00,0.000000,0.000000,1.0,7,0.611332,True,REGULAR,USD,call,2023-06-09 23:59:59,7
9,JPM230609C00127000,2023-06-02 14:06:25+00:00,127.0,13.13,13.15,13.85,3.730001,39.680855,1.0,1,0.529302,True,REGULAR,USD,call,2023-06-09 23:59:59,7


In [ ]:
import time

underlying = 'JPM'




for row in range(df.shape[0]):
  print(row, 'out of 1021', end=' ')

  time.sleep(3) # Sleep for 3 seconds

  date_ = str(df.iloc[row]['expDate']).split('-')
  y, m, d = int(date_[0]), int(date_[1]), int(date_[2].split()[0])

  strike_ = df.iloc[row]['strike']
  option_type = df.iloc[row]['optionType']


  
  prices = get_prices(y, m, d, strike_, underlying, option_type)
  print('timeseries size is', prices.shape[0])

By finging needed timeseries lenght acessing 

In [12]:
def get_prices(y, m, d, strike_, underlying, option_type):
  with client.connect():  # Make any requests for data inside this block. Requests made outside this block wont run.
      if option_type == 'call':
        right_ = OptionRight.CALL
      else:
        right_ = OptionRight.PUT

      data = client.get_hist_option(
            req=OptionReqType.EOD,
            root = underlying,
            exp = date(y, m, d),
            strike = strike_,
            right = right_,
            date_range=DateRange(date(2020, 1, 1), date(2023, 6, 30))
    
      )
      return data

In [13]:
prices = get_prices(2024, 1, 19, 150, 'JPM', 'call') # get_prices(y, m, d, strike_, underlying, option_type)


In [14]:
prices.shape

(398, 7)

In [17]:
prices.head(5)

,DataType.OPEN,DataType.HIGH,DataType.LOW,DataType.CLOSE,DataType.VOLUME,DataType.COUNT,DataType.DATE
0,26.12,26.12,26.12,26.12,1,1,2021-09-13
1,27.86,27.89,25.20,25.20,48,7,2021-09-14
2,26.00,26.00,26.00,26.00,1,1,2021-09-16
3,22.60,22.65,22.60,22.65,2,2,2021-09-20
4,23.21,23.38,23.21,23.38,12,3,2021-09-21


In [15]:
prices.info

<bound method DataFrame.info of      DataType.OPEN  DataType.HIGH  DataType.LOW  DataType.CLOSE  \
0            26.12          26.12         26.12           26.12   
1            27.86          27.89         25.20           25.20   
2            26.00          26.00         26.00           26.00   
3            22.60          22.65         22.60           22.65   
4            23.21          23.38         23.21           23.38   
..             ...            ...           ...             ...   
393           5.70           6.00          5.45            5.80   
394           5.60           5.90          5.40            5.65   
395           5.30           5.35          4.82            5.10   
396           5.20           5.80          5.16            5.42   
397           5.90           6.85          5.90            6.69   

     DataType.VOLUME  DataType.COUNT DataType.DATE  
0                  1               1    2021-09-13  
1                 48               7    2021-09-14  
2   

In [16]:
prices.describe

<bound method NDFrame.describe of      DataType.OPEN  DataType.HIGH  DataType.LOW  DataType.CLOSE  \
0            26.12          26.12         26.12           26.12   
1            27.86          27.89         25.20           25.20   
2            26.00          26.00         26.00           26.00   
3            22.60          22.65         22.60           22.65   
4            23.21          23.38         23.21           23.38   
..             ...            ...           ...             ...   
393           5.70           6.00          5.45            5.80   
394           5.60           5.90          5.40            5.65   
395           5.30           5.35          4.82            5.10   
396           5.20           5.80          5.16            5.42   
397           5.90           6.85          5.90            6.69   

     DataType.VOLUME  DataType.COUNT DataType.DATE  
0                  1               1    2021-09-13  
1                 48               7    2021-09-14  
2 

In [ ]:
prices.to_csv('JPM240119C00150000.csv', index=False)